# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

## Set Hyperparameters

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [4]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../../data/preprocessed_v2/Tnoise_comp_mecab_1073.csv') # processed된 파일까지 경로 
TEST_DIR = os.path.join(BASE_DIR, '../../data/preprocessed_v2/Lnoise_1200.csv')
train_name = os.path.splitext(os.path.basename(DATA_DIR))[0]  # processed된 파일 이름 추출
OUTPUT_DIR = os.path.join(BASE_DIR, '../../data/preprocessed_v2/Lnoise_comp_mecab_1200_v1') # 해당 파일 이름으로 output 폴더 생성

## Load Tokenizer and Model

In [5]:
model_name = "classla/multilingual-IPTC-news-topic-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=7,
    ignore_mismatched_sizes=True  # 크기가 맞지 않는 레이어를 무시하고 로드
).to(DEVICE)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at classla/multilingual-IPTC-news-topic-classifier and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([17]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([17, 1024]) in the checkpoint and torch.Size([7, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
data = pd.read_csv(DATA_DIR)
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=SEED)
# dataset_train = data

## Define Dataset

In [7]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [8]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [10]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [11]:
### for wandb setting
#os.environ['WANDB_DISABLED'] = 'true'

In [12]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=10,
    eval_steps=10,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 8e-06, # suggested
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32, # suggested
    per_device_eval_batch_size=32,
    num_train_epochs=5, # suggested
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1
10,1.907700,1.783986,0.276444
20,1.819000,1.731059,0.309728
30,1.763300,1.616942,0.337185
40,1.683900,1.554862,0.443594
50,1.591000,1.501784,0.466420
60,1.523700,1.471366,0.474996
70,1.518800,1.440975,0.498851
80,1.488800,1.414121,0.549297
90,1.459900,1.401394,0.551782
100,1.428500,1.389981,0.546529


TrainOutput(global_step=135, training_loss=1.5695234687240036, metrics={'train_runtime': 146.8905, 'train_samples_per_second': 29.205, 'train_steps_per_second': 0.919, 'total_flos': 999513272609280.0, 'train_loss': 1.5695234687240036, 'epoch': 5.0})

## Evaluate Model

In [15]:
dataset_test = pd.read_csv(TEST_DIR)

In [16]:
model.eval()
preds = []

for idx, sample in tqdm(dataset_test.iterrows(), total=len(dataset_test), desc="Evaluating"):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

Evaluating:   0%|          | 0/1200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 1200/1200 [00:35<00:00, 33.62it/s]


In [17]:
dataset_test['target'] = preds
dataset_test.to_csv(OUTPUT_DIR+'.csv', index=False) # processed된 파일 이름 폴더에 결과 저장

In [18]:
final_Tnoise = pd.read_csv(DATA_DIR)
final_Lnoise = pd.read_csv(OUTPUT_DIR+'.csv')

clear = pd.concat([final_Tnoise, final_Lnoise], ignore_index=True)
clear.to_csv('../../data/preprocessed_v2/clear_comp_mecab_2273.csv', index=False) # 최종 결과 저장

print(clear.shape)

(2273, 3)
